In [1]:
from astropy.io import fits

# Open the file
with fits.open('jw01366-o004_t001_nirspec_clear-prism-s1600a1-sub512_x1dints.fits') as hdul:
    # List all extensions to see structure
    hdul.info()
    # This will show you what sections exist

Filename: jw01366-o004_t001_nirspec_clear-prism-s1600a1-sub512_x1dints.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     362   ()      
  1  INT_TIMES     1 BinTableHDU     24   21500R x 7C   [J, D, D, D, D, D, D]   
  2  EXTRACT1D     1 BinTableHDU    122   6100R x 27C   [J, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432J, 432D, 432D, 432D, 432D, 432D, 432D, J, J, D, D, D, D, D, D]   
  3  EXTRACT1D     2 BinTableHDU    122   6100R x 27C   [J, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432J, 432D, 432D, 432D, 432D, 432D, 432D, J, J, D, D, D, D, D, D]   
  4  EXTRACT1D     3 BinTableHDU    122   6100R x 27C   [J, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432J, 432D, 432D, 432D, 432D, 432D, 432D, J, J, D, D, D, D, D, D]   
  5  EXTRACT1D     4 BinTableHDU    122   3200R x 27C   [J, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432J, 432D, 432D, 4

In [3]:
"""
Extract all EXTRACT1D versions from JWST WASP-39b NIRSpec PRISM data
Converts FITS data to CSV format for each detector/version
"""

import pandas as pd
import numpy as np
from astropy.io import fits
import os
from pathlib import Path

def extract_fits_to_csv(fits_file, output_dir='extracted_data'):
    """
    Extract all EXTRACT1D versions from FITS file and save as CSV
    
    Parameters
    ----------
    fits_file : str
        Path to the FITS file
    output_dir : str
        Directory to save CSV files
    """
    
    # Create output directory
    Path(output_dir).mkdir(exist_ok=True)
    
    print(f"Opening FITS file: {fits_file}")
    
    with fits.open(fits_file) as hdul:
        # Print file structure
        print("\n" + "="*70)
        print("FITS FILE STRUCTURE")
        print("="*70)
        hdul.info()
        
        # Extract INT_TIMES (integration timing information)
        print("\n" + "="*70)
        print("EXTRACTING INTEGRATION TIMES")
        print("="*70)
        int_times_data = hdul[1].data
        int_times_df = pd.DataFrame(int_times_data)
        int_times_file = os.path.join(output_dir, 'integration_times.csv')
        int_times_df.to_csv(int_times_file, index=False)
        print(f"Saved: {int_times_file}")
        print(f"Shape: {int_times_df.shape}")
        print(f"Columns: {list(int_times_df.columns)}")
        
        # Extract each EXTRACT1D version
        print("\n" + "="*70)
        print("EXTRACTING SPECTRA FROM ALL VERSIONS")
        print("="*70)
        
        extract_hdu_indices = [2, 3, 4, 5]  # EXTRACT1D HDUs
        
        for version_num, hdu_idx in enumerate(extract_hdu_indices, start=1):
            try:
                hdu = hdul[hdu_idx]
                data = hdu.data
                header = hdu.header
                
                print(f"\n--- EXTRACT1D Version {version_num} (HDU {hdu_idx}) ---")
                print(f"Rows: {len(data)}, Columns: {len(data.dtype.names)}")
                
                # Get column names from FITS
                col_names = data.dtype.names
                print(f"Column names: {col_names[:5]}... (showing first 5)")
                
                # Create DataFrame
                df_dict = {}
                df_dict['integration_id'] = np.arange(len(data))
                
                # Extract each column
                for col_name in col_names:
                    col_data = data[col_name]
                    
                    # If column is array-like (wavelengths, flux, errors)
                    if len(col_data.shape) > 1:
                        n_wavelengths = col_data.shape[1]
                        for i in range(n_wavelengths):
                            df_dict[f'{col_name}_{i}'] = col_data[:, i]
                    else:
                        # Scalar column
                        df_dict[col_name] = col_data
                
                df = pd.DataFrame(df_dict)
                
                # Save to CSV
                output_file = os.path.join(output_dir, f'extract1d_version_{version_num}.csv')
                df.to_csv(output_file, index=False)
                print(f"Saved: {output_file}")
                print(f"Shape: {df.shape}")
                print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
                
            except IndexError:
                print(f"HDU {hdu_idx} not found, skipping...")
            except Exception as e:
                print(f"Error processing HDU {hdu_idx}: {e}")
    
    print("\n" + "="*70)
    print("EXTRACTION COMPLETE")
    print("="*70)
    print(f"All files saved to: {output_dir}/")
    print("\nGenerated files:")
    for file in sorted(os.listdir(output_dir)):
        filepath = os.path.join(output_dir, file)
        file_size = os.path.getsize(filepath) / 1024**2
        print(f"  - {file} ({file_size:.2f} MB)")


def load_and_inspect_csv(csv_file):
    """
    Load and inspect a generated CSV file
    
    Parameters
    ----------
    csv_file : str
        Path to CSV file
    """
    df = pd.read_csv(csv_file)
    print(f"\nInspecting: {csv_file}")
    print(f"Shape: {df.shape}")
    print(f"\nFirst few rows:")
    print(df.head())
    print(f"\nColumn names (first 10):")
    print(list(df.columns)[:10])
    print(f"\nData types:")
    print(df.dtypes.value_counts())


def combine_all_versions(output_dir='extracted_data', combined_file='combined_spectrum.csv'):
    """
    Combine all extracted versions into a single master file
    
    Parameters
    ----------
    output_dir : str
        Directory containing extracted CSV files
    combined_file : str
        Output filename for combined data
    """
    print("\n" + "="*70)
    print("COMBINING ALL VERSIONS")
    print("="*70)
    
    all_dfs = []
    
    for version_num in range(1, 5):
        csv_file = os.path.join(output_dir, f'extract1d_version_{version_num}.csv')
        try:
            df = pd.read_csv(csv_file)
            df['source_version'] = version_num  # Add version tracking
            all_dfs.append(df)
            print(f"Loaded version {version_num}: {df.shape}")
        except FileNotFoundError:
            print(f"Version {version_num} file not found, skipping...")
    
    if all_dfs:
        combined_df = pd.concat(all_dfs, axis=0, ignore_index=True)
        output_path = os.path.join(output_dir, combined_file)
        combined_df.to_csv(output_path, index=False)
        print(f"\nCombined file saved: {output_path}")
        print(f"Combined shape: {combined_df.shape}")
        print(f"Versions included: {sorted(combined_df['source_version'].unique())}")


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Set the path to your FITS file
    fits_file = 'jw01366-o004_t001_nirspec_clear-prism-s1600a1-sub512_x1dints.fits'
    output_directory = 'wasp39_extracted_data'
    
    # Check if file exists
    if not os.path.exists(fits_file):
        print(f"Error: FITS file not found: {fits_file}")
        print("Please download the file from MAST or adjust the path")
    else:
        # Extract all versions
        extract_fits_to_csv(fits_file, output_directory)
        
        # Optional: Inspect one of the files
        print("\n" + "="*70)
        print("SAMPLE INSPECTION")
        print("="*70)
        sample_csv = os.path.join(output_directory, 'extract1d_version_1.csv')
        if os.path.exists(sample_csv):
            load_and_inspect_csv(sample_csv)
        
        # Optional: Combine all versions
        combine_all_versions(output_directory)

Opening FITS file: jw01366-o004_t001_nirspec_clear-prism-s1600a1-sub512_x1dints.fits

FITS FILE STRUCTURE
Filename: jw01366-o004_t001_nirspec_clear-prism-s1600a1-sub512_x1dints.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     362   ()      
  1  INT_TIMES     1 BinTableHDU     24   21500R x 7C   [J, D, D, D, D, D, D]   
  2  EXTRACT1D     1 BinTableHDU    122   6100R x 27C   [J, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432J, 432D, 432D, 432D, 432D, 432D, 432D, J, J, D, D, D, D, D, D]   
  3  EXTRACT1D     2 BinTableHDU    122   6100R x 27C   [J, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432J, 432D, 432D, 432D, 432D, 432D, 432D, J, J, D, D, D, D, D, D]   
  4  EXTRACT1D     3 BinTableHDU    122   6100R x 27C   [J, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432D, 432J, 432D, 432D, 432D, 432D, 432D, 432D, J, J, D, D, D, D, D, D]   
  5  EXTRACT1D     4 BinTableHDU    122